# 🧪LAB 4 – Knowledge Base + MCP filesystem

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

GITHUB_ENDPOINT = os.getenv("GITHUB_ENDPOINT")
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
GITHUB_MODEL = os.getenv("GITHUB_MODEL")

if not all([GITHUB_ENDPOINT, GITHUB_TOKEN, GITHUB_MODEL]):
    raise RuntimeError(
        "❌ Faltan variables en .env. Asegúrate de tener "
        "GITHUB_ENDPOINT, GITHUB_TOKEN y GITHUB_MODEL configuradas."
    )

os.environ["OPENAI_API_KEY"] = GITHUB_TOKEN
os.environ["OPENAI_BASE_URL"] = GITHUB_ENDPOINT
os.environ["OPENAI_CHAT_MODEL_ID"] = GITHUB_MODEL

print("✅ Configuración cargada desde .env")
print("  ENDPOINT:", GITHUB_ENDPOINT)
print("  MODEL:", GITHUB_MODEL)


✅ Configuración cargada desde .env
  ENDPOINT: https://models.github.ai/inference
  MODEL: gpt-4o


In [2]:
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient

import asyncio
import json
from pathlib import Path

DATA_DIR = Path("data")
OUT_DIR = Path("out")
KB_DIR = Path("kb")

DATA_DIR.mkdir(exist_ok=True)
OUT_DIR.mkdir(exist_ok=True)
KB_DIR.mkdir(exist_ok=True)

print("📂 DATA_DIR:", DATA_DIR.resolve())
print("📂 OUT_DIR:", OUT_DIR.resolve())
print("📂 KB_DIR:", KB_DIR.resolve())


📂 DATA_DIR: C:\Repos\agent-framework-workshop\data
📂 OUT_DIR: C:\Repos\agent-framework-workshop\out
📂 KB_DIR: C:\Repos\agent-framework-workshop\kb


In [3]:
kb_files = list(KB_DIR.glob("*.md"))
print("📄 Ficheros en kb/:")
for f in kb_files:
    print(" -", f.name)

if not kb_files:
    print("⚠️ No hay ficheros en kb/. Recuerda crear politica_portatiles.md, politica_vacaciones.md, faq_it.md, faq_rrhh.md")


📄 Ficheros en kb/:
 - faq_it.md
 - faq_rrhh.md
 - politica_portatiles.md
 - politica_vacaciones.md


In [4]:
from typing import List, Dict, Annotated
from pydantic import Field

def kb_list_files() -> List[str]:
    """
    Devuelve la lista de nombres de ficheros .md disponibles en la KB local.
    Equivalente a un 'list' del MCP filesystem sobre kb/.
    """
    files = [f.name for f in KB_DIR.glob("*.md")]
    return files

def kb_read_file(
    filename: Annotated[str, Field(description="Nombre de un fichero .md dentro de la carpeta kb/")]
) -> str:
    """
    Lee el contenido completo de un fichero de la KB.
    """
    path = KB_DIR / filename
    if not path.exists():
        raise FileNotFoundError(f"No se encontró el fichero {filename} en kb/.")
    return path.read_text(encoding="utf-8")


## KnowledgeAgent: agente especializado en leer la KB

Este agente:

Usa kb_list_files y kb_read_file.

Dado un mensaje del usuario, decide qué doc abrir y responde con un resumen centrado en la pregunta.

In [5]:
KNOWLEDGE_INSTRUCTIONS = """
Eres un agente especializado en consultar la base de conocimiento interna (KB).

Dispones de las funciones:
- kb_list_files(): para ver qué documentos hay en la KB.
- kb_read_file(filename): para leer el contenido de un documento .md en texto.

Tu trabajo es:
1. Leer la pregunta del usuario.
2. Decidir qué documentos de la KB pueden contener la respuesta (por ejemplo: politica_portatiles.md, politica_vacaciones.md, faq_it.md, faq_rrhh.md).
3. Usar kb_read_file para leer solo los documentos relevantes.
4. Responder en español, citando solo información que aparezca en la documentación cargada.
5. Si no encuentras la respuesta en la KB, dilo explícitamente y sugiere crear un ticket al Service Desk.

Reglas:
- No inventes políticas ni normas que no estén en la documentación.
- Intenta ser concreto: cita secciones o puntos relevantes.
- Si la pregunta es ambigua, pide aclaraciones.
"""

async def create_knowledge_agent() -> ChatAgent:
    chat_client = OpenAIChatClient()
    agent = chat_client.create_agent(
        name="KnowledgeAgent",
        instructions=KNOWLEDGE_INSTRUCTIONS,
        tools=[kb_list_files, kb_read_file],
    )
    return agent


In [7]:
async def demo_knowledge_agent():
    agent = await create_knowledge_agent()
    thread = agent.get_new_thread()

    pregunta = "¿Puedo pedir un segundo portátil según la política de la empresa?"
    print("👤 Usuario:", pregunta, "\n")

    result = await agent.run(pregunta, thread=thread)
    print("🤖 KnowledgeAgent:\n")
    print(result.text)

await demo_knowledge_agent()


👤 Usuario: ¿Puedo pedir un segundo portátil según la política de la empresa? 

🤖 KnowledgeAgent:

De acuerdo con la **Política de Asignación de Portátiles**, como norma general solo se asigna **un único portátil por persona**. Sin embargo, en situaciones excepcionales es posible solicitar un segundo portátil en los siguientes casos:

1. Si realizas desplazamientos internacionales de forma frecuente (al menos una vez al mes).
2. Si tienes limitaciones físicas que requieran un equipo adicional adaptado.
3. Otros casos especiales que sean aprobados por el Director de IT.

Para estas excepciones, es imprescindible:
- Justificar por escrito la necesidad del segundo portátil.
- Obtener la aprobación del responsable directo.
- Contar con la revisión de RRHH y Dirección de IT, quienes tienen la facultad de retirar equipos adicionales en cualquier momento.

Si cumples con alguno de estos criterios, deberás documentar la justificación y presentar la solicitud mediante el proceso establecido en l

In [10]:
async def responder_con_kb_y_service_desk(email_usuario: str, texto_usuario: str):
    """
    Orquesta:
    1. Intenta responder con KnowledgeAgent (KB).
    2. Si no parece resoluble solo con KB, pasa al ServiceDeskAgentMemory.
    """
    knowledge_agent = await create_knowledge_agent()
    service_agent = await create_service_desk_agent_with_memory()  # del LAB 3
    thread_kb = knowledge_agent.get_new_thread()
    thread_sd = service_agent.get_new_thread()

    # 1) Preguntamos primero a la KB
    print("=== Paso 1: Consultando KB ===")
    kb_result = await knowledge_agent.run(texto_usuario, thread=thread_kb)
    respuesta_kb = kb_result.text.strip()
    print("\n🤖 KnowledgeAgent:\n", respuesta_kb, "\n")

    # Heurística simple: si KB dice claramente que no encuentra nada, pasamos a Service Desk
    # (esto lo puedes afinar con un prompt más estructurado, tags, etc.)
    lower = respuesta_kb.lower()
    kb_util = not (
        "no encuentro" in lower or
        "no aparece en la documentación" in lower or
        "no está en la base de conocimiento" in lower
    )

    if kb_util:
        print("✅ La KB parece suficiente para responder. No creamos ticket.")
        return respuesta_kb

    print("=== Paso 2: Delegando al Service Desk (con memoria) ===")
    result_sd, _ = await run_with_user_profile(service_agent, email_usuario, texto_usuario, thread_sd)
    respuesta_sd = result_sd.text.strip()
    print("\n🤖 ServiceDeskAgentMemory:\n", respuesta_sd, "\n")
    return respuesta_sd


Esto es un “proto-multiagente”: KB y Service Desk colaboran, pero la orquestación la haces tú con Python. En el siguiente nivel podrías hacer que el propio ServiceDeskAgentMemory tuviera tools para llamar a KB (A2A / MCP real).

In [14]:
SERVICE_DESK_MEMORY_INSTRUCTIONS = """
Eres un agente de Service Desk interno con capacidad de recordar preferencias del usuario.

Tu trabajo es:
1. Entender la solicitud del usuario en lenguaje natural.
2. Consultar el perfil del usuario (que se te pasa en el mensaje como JSON) y usarlo como contexto:
   - sede
   - preferencias de portátil (preferencias_equipo)
   - idioma_respuesta
3. Decidir si debe crearse un ticket nuevo o modificarse uno existente.
4. Si procede, llamar a las funciones disponibles:
   - `crear_ticket_fc` para registrar un nuevo ticket.
   - `actualizar_estado_ticket_fc` para cambiar el estado de un ticket existente.
   - `actualizar_perfil_usuario_fc` para actualizar las preferencias del usuario cuando te lo pida.
5. Responder al usuario confirmando la acción realizada (id del ticket, nuevo estado, etc.).

Reglas:
- Si el usuario da información de perfil (“trabajo en Barcelona”, “prefiero portátiles Lenovo”), deberías llamar a `actualizar_perfil_usuario_fc`.
- Cuando crees un ticket, si conoces la sede o preferencia de portátil del usuario, intenta incluirlo en el detalle del ticket.
- Pregunta por el email del usuario si no se ha proporcionado antes de operaciones críticas (crear ticket, guardar perfil).
- No inventes identificadores ni emails.
- Usa estados coherentes: pendiente, en_progreso, resuelto_auto, resuelto_humano, cancelado, etc.
"""


In [16]:
def crear_ticket_fc(
    desde_email: Annotated[str, Field(description="Dirección de correo del solicitante")],
    departamento: Annotated[str, Field(description="Departamento destino: IT, RRHH, Facilities u otro")],
    categoria: Annotated[str, Field(description="Categoria de la solicitud: nuevo_equipo, incidencia, vacaciones, certificado, mantenimiento, otro")],
    prioridad: Annotated[str, Field(description="Prioridad: alta, media o baja")],
    resumen: Annotated[str, Field(description="Resumen corto de la solicitud")],
    detalle: Annotated[str, Field(description="Detalle extendido de la solicitud")]
) -> Dict:
    """
    Crea un nuevo ticket en tickets.csv y guarda un JSON en out/ticket_{id}.json.
    Devuelve el diccionario con los datos completos del ticket creado.
    """
    data = load_tickets()

    if data.empty:
        next_id = 1
    else:
        ids = pd.to_numeric(data["id"], errors="coerce")
        next_id = int(ids.max() or 0) + 1

    nuevo_ticket = {
        "id": str(next_id),
        "fecha": datetime.now().strftime("%Y-%m-%d"),
        "solicitante": desde_email,
        "departamento": departamento or "Otro",
        "categoria": categoria or "otro",
        "prioridad": prioridad or "media",
        "estado": "pendiente",
        "resumen": (resumen or "")[:200],
        "detalle": detalle or "",
    }

    data = pd.concat([data, pd.DataFrame([nuevo_ticket])], ignore_index=True)
    data.to_csv(TICKETS_CSV, index=False, encoding="utf-8")

    ticket_json_path = OUT_DIR / f"ticket_{next_id}.json"
    with ticket_json_path.open("w", encoding="utf-8") as f:
        json.dump(nuevo_ticket, f, ensure_ascii=False, indent=2)

    print(f"✅ [Tool] Ticket creado con id={next_id}")
    return nuevo_ticket


def actualizar_estado_ticket_fc(
    id_ticket: Annotated[int, Field(description="Identificador numérico del ticket a actualizar")],
    nuevo_estado: Annotated[str, Field(description="Nuevo estado del ticket, por ejemplo: pendiente, en_progreso, resuelto_auto, resuelto_humano, cancelado")]
) -> Dict:
    """
    Actualiza el campo 'estado' de un ticket en tickets.csv y en el JSON correspondiente (si existe).
    Devuelve el ticket actualizado, o un mensaje de error si el id no existe.
    """
    data = load_tickets()
    if data.empty:
        return {
            "ok": False,
            "mensaje": f"No hay tickets en el sistema. No se encontró el ticket con id={id_ticket}."
        }

    mask = data["id"] == str(id_ticket)
    if not mask.any():
        return {
            "ok": False,
            "mensaje": f"No se encontró el ticket con id={id_ticket}."
        }

    data.loc[mask, "estado"] = nuevo_estado
    data.to_csv(TICKETS_CSV, index=False, encoding="utf-8")

    ticket_actualizado = data.loc[mask].iloc[0].to_dict()

    ticket_json_path = OUT_DIR / f"ticket_{id_ticket}.json"
    if ticket_json_path.exists():
        with ticket_json_path.open("w", encoding="utf-8") as f:
            json.dump(ticket_actualizado, f, ensure_ascii=False, indent=2)

    print(f"✅ [Tool] Estado del ticket {id_ticket} actualizado a '{nuevo_estado}'")
    return {
        "ok": True,
        "ticket": ticket_actualizado
    }


In [18]:
def actualizar_perfil_usuario_fc(
    email: Annotated[str, Field(description="Email corporativo del usuario para guardar su perfil")],
    sede: Annotated[str, Field(description="Ciudad o sede principal de trabajo del usuario")] = "",
    preferencias_equipo: Annotated[str, Field(description="Preferencia de marca o tipo de portátil (p.ej. Lenovo, Dell, etc.)")] = "",
    idioma_respuesta: Annotated[str, Field(description="Idioma preferido para las respuestas (es, en, etc.)")] = ""
) -> Dict:
    """
    Actualiza el perfil de usuario (sede, preferencias de equipo, idioma) en profiles.json.
    Devuelve el perfil completo después de la actualización.
    """
    perfil = update_user_profile(
        email=email,
        sede=sede,
        preferencias_equipo=preferencias_equipo,
        idioma_respuesta=idioma_respuesta
    )
    print(f"✅ [Tool] Perfil actualizado para {email}: {perfil}")
    return perfil


In [12]:
from agent_framework import ChatAgent

async def create_service_desk_agent_with_memory() -> ChatAgent:
    """
    Agente de Service Desk que:
    - Usa function calling para tickets (crear y actualizar estado).
    - Usa function calling para actualizar el perfil del usuario.
    """
    chat_client = OpenAIChatClient()
    agent = chat_client.create_agent(
        name="ServiceDeskAgentMemory",
        instructions=SERVICE_DESK_MEMORY_INSTRUCTIONS,
        tools=[crear_ticket_fc, actualizar_estado_ticket_fc, actualizar_perfil_usuario_fc],
    )
    return agent

In [19]:
async def demo_kb_mas_service_desk():
    email = "antonio.soto@empresa.local"

    # Caso 1: pregunta 100% de política, debería resolverse solo con KB
    texto1 = "¿Cuántos días de vacaciones tengo según la política?"
    print("👤 Usuario:", texto1, "\n")
    resp1 = await responder_con_kb_y_service_desk(email, texto1)
    print("💬 Respuesta final:\n", resp1, "\n")
    print("="*80, "\n")

    # Caso 2: algo que podría dar lugar a ticket
    texto2 = "Quiero pedir mis vacaciones del 1 al 15 de agosto."
    print("👤 Usuario:", texto2, "\n")
    resp2 = await responder_con_kb_y_service_desk(email, texto2)
    print("💬 Respuesta final:\n", resp2, "\n")

await demo_kb_mas_service_desk()


👤 Usuario: ¿Cuántos días de vacaciones tengo según la política? 

=== Paso 1: Consultando KB ===

🤖 KnowledgeAgent:
 Según la política de vacaciones:

- Tienes derecho a **23 días laborables de vacaciones al año**.
- Estos días son adicionales a los festivos nacionales, autonómicos y locales.
- Si tienes más de **5 años de antigüedad**, tendrás 1 día adicional (24 días).
- Si tienes más de **10 años de antigüedad**, tendrás 2 días adicionales (25 días).

Si necesitas más información o aclaraciones, puedes solicitarla directamente a Recursos Humanos o mediante un ticket al Service Desk en la categoría `RRHH/vacaciones`. 

✅ La KB parece suficiente para responder. No creamos ticket.
💬 Respuesta final:
 Según la política de vacaciones:

- Tienes derecho a **23 días laborables de vacaciones al año**.
- Estos días son adicionales a los festivos nacionales, autonómicos y locales.
- Si tienes más de **5 años de antigüedad**, tendrás 1 día adicional (24 días).
- Si tienes más de **10 años de a

## MCP

In [23]:
from agent_framework import MCPStdioTool

# Directorio donde está nuestra KB (ya definido en el LAB 4)
KB_DIR = Path("kb")
KB_DIR.mkdir(exist_ok=True)

print("📂 KB_DIR:", KB_DIR.resolve())

async def create_filesystem_mcp_tool():
    """
    Crea un MCPStdioTool que lanza un servidor MCP de filesystem
    apuntando a la carpeta kb/, sin usar Docker.
    
    Requiere tener instalado:
    - uv / uvx
    - el paquete mcp-server-filesystem (o equivalente)
    """

    # MCPStdioTool ejecutará: uvx mcp-server-filesystem
    mcp_tool = MCPStdioTool(
        name="filesystem",
        command="uvx",
        args=["mcp-server-filesystem"]
    )

    await mcp_tool.connect()  # muy importante: conectar antes de usar
    return mcp_tool


📂 KB_DIR: C:\Repos\agent-framework-workshop\kb


In [21]:
# KnowledgeAgent con MCP filesystem tool
KNOWLEDGE_INSTRUCTIONS = """
Eres un agente especializado en consultar la base de conocimiento interna (KB).

Dispones de herramientas MCP expuestas por un servidor de filesystem:
- Herramientas para listar ficheros disponibles.
- Herramientas para leer el contenido de ficheros .md.

Tu trabajo es:
1. Leer la pregunta del usuario.
2. Identificar qué ficheros de la KB pueden contener la respuesta.
3. Usar las herramientas MCP para leer solo los documentos relevantes.
4. Responder en español, citando solo información que aparezca en esos documentos.
5. Si no encuentras la respuesta en la KB, dilo explícitamente y sugiere crear un ticket al Service Desk.

Reglas:
- No inventes políticas ni normas que no estén en la documentación.
- Sé concreto y menciona los puntos clave.
- Si la pregunta es ambigua, pide aclaraciones.
"""

async def create_knowledge_agent_mcp() -> ChatAgent:
    """
    Crea un KnowledgeAgent que usa MCPStdioTool (filesystem) como source de tools.
    """
    chat_client = OpenAIChatClient()

    # 1) Crear y conectar MCPStdioTool (filesystem)
    filesystem_mcp = await create_filesystem_mcp_tool()

    # 2) Enumerar las funciones expuestas por el servidor MCP
    #    (cada función MCP se convierte en una tool callable por el LLM)
    mcp_tools = [*filesystem_mcp.functions]

    # 3) Crear el agente con esas tools
    agent = ChatAgent(
        chat_client=chat_client,
        name="KnowledgeAgentMCP",
        instructions=KNOWLEDGE_INSTRUCTIONS,
        tools=mcp_tools,
    )

    # IMPORTANTE: devolvemos también el mcp para poder cerrarlo luego
    return agent, filesystem_mcp


In [26]:
async def demo_knowledge_agent_mcp():
    agent, filesystem_mcp = await create_knowledge_agent_mcp()
    thread = agent.get_new_thread()

    try:
        pregunta = "¿Puedo pedir un segundo portátil según la política de la empresa?"
        print("👤 Usuario:", pregunta, "\n")

        result = await agent.run(pregunta, thread=thread)
        print("🤖 KnowledgeAgentMCP:\n")
        print(result.text)
    finally:
        # Cerrar la conexión con el servidor MCP
        await filesystem_mcp.disconnect()

await demo_knowledge_agent_mcp()


ToolException: Failed to connect to the MCP server. Please check your configuration.